# ALGORITMO DOWNLOAD PCAP

Il download è suddiviso in 5 fasi e sono il frutto di un processo di reverse engineering del processo di download della pagina contenente i pcap files (<a href='http://dataset.tlm.unavarra.es/ransomware/'>dataset.tlm.unavarra.es</a>).

1. ristrutturare json estratto dal sito;
2. download pcap files da <a href='https://www.malware-traffic-analysis.net/index.html'>malware-traffic-analysis</a>;
3. invio POST dataset.tlm.unavarra.es/ransomware/php/descargar.php;
4. acquisire risultato POST;
5. download pcap files da <a href='http://dataset.tlm.unavarra.es/download/'>dataset.tlm.unavarra.es/download/</a>.

In [5]:
import os
import json

## RISTRUTTURAZIONE JSON

Il file 'ransom.json' è stato estratto nel momento in cui si carica la pagina il quale contiene un elemento per ciascun sample.
<br>
<img src="imgMD/response_json_php.png" width="850"/>
<br>
Alla fine di questo processo si vuole ottenere sempre un **json ristrutturato** con la seguente struttura in modo da raggruppare i vari sample per classe per poter replicare la struttura all'interno del file system:

~~~
{
    "famiglia1":[
        {
        "SamplePcap": "path",
        "Hash": "file_hash",
        "Link": "URL",
        "Scenario": "NAT o Original"
        },
        {"..."}
    ],

    "faliglia2": ["..."],

    "..." : ["..."],

    "faligliaN": ["..."]
}
~~~

In [8]:
def serialize_json(filename, data):
    with open(f"{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

In [ ]:
samples = "ransom.json"
samples_dict = read_json(samples)
samples_restructured = {}

for i in range(len(samples_dict)):
    family = samples_dict[str(i)][0]['Family']

    if family not in samples_restructured.keys():
        samples_restructured[family] = []

    sample = {
        "SamplePcap" : samples_dict[str(i)][0]['SamplePcap'],
        "Hash" : samples_dict[str(i)][0]['Hash'],
        "Link" : samples_dict[str(i)][0]['Link'],
        "Scenario" : samples_dict[str(i)][0]['Scenario']
    }

    samples_restructured[family].append(sample)

serialize_json('ransom_restructured.json', samples_restructured)